In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

import random
import gc

In [2]:
import _pickle as pickle

def save(file,name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'wb')
    else:
        outfile = open(name+'.pickle', 'wb')
    pickle.dump(file, outfile)
    outfile.close
    
def load(name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'rb')
    else:
        outfile = open(name+'.pickle', 'rb')
    file = pickle.load(outfile)
    outfile.close
    return file

In [20]:
sales = pd.read_csv('sales_train_validation.csv')
calendar = pd.read_csv('calendar.csv')
prices = pd.read_csv('sell_prices.csv')
submission = pd.read_csv('sample_submission.csv')

prices['id'] = prices['item_id'] + '_' + prices['store_id'] +'_'+ 'validation'

In [68]:
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [76]:
calendar['d'].unique()

array(['d_1', 'd_2', 'd_3', ..., 'd_1967', 'd_1968', 'd_1969'],
      dtype=object)

In [75]:
prices['wm_yr_wk']

0          11325
1          11326
2          11327
3          11328
4          11329
           ...  
6841116    11617
6841117    11618
6841118    11619
6841119    11620
6841120    11621
Name: wm_yr_wk, Length: 6841121, dtype: int64

In [47]:
sales

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
id,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1_validation,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
HOBBIES_1_002_CA_1_validation,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
HOBBIES_1_003_CA_1_validation,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
HOBBIES_1_004_CA_1_validation,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
HOBBIES_1_005_CA_1_validation,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FOODS_3_823_WI_3_validation,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
FOODS_3_824_WI_3_validation,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
FOODS_3_825_WI_3_validation,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0


In [77]:
prices

,store_id,item_id,wm_yr_wk,sell_price,id
0,CA_1,HOBBIES_1_001,11325,9.58,HOBBIES_1_001_CA_1_validation
1,CA_1,HOBBIES_1_001,11326,9.58,HOBBIES_1_001_CA_1_validation
2,CA_1,HOBBIES_1_001,11327,8.26,HOBBIES_1_001_CA_1_validation
3,CA_1,HOBBIES_1_001,11328,8.26,HOBBIES_1_001_CA_1_validation
4,CA_1,HOBBIES_1_001,11329,8.26,HOBBIES_1_001_CA_1_validation
...,...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.00,FOODS_3_827_WI_3_validation
6841117,WI_3,FOODS_3_827,11618,1.00,FOODS_3_827_WI_3_validation
6841118,WI_3,FOODS_3_827,11619,1.00,FOODS_3_827_WI_3_validation
6841119,WI_3,FOODS_3_827,11620,1.00,FOODS_3_827_WI_3_validation


In [79]:
prices['id'].unique().shape

(30490,)

In [80]:
prices['wm_yr_wk'].unique().shape

(282,)

In [81]:
282*30490

8598180

In [22]:
dico_id = {}

sales.index = sales['id']

for elt in tqdm(sales['id'].unique()):
    a = sales.loc[elt]
    
    item_id = a['item_id']
    dept_id = a['dept_id']
    store_id = a['store_id']
    cat_id = a['cat_id']
    state_id = a['state_id']
    
    dico_id[elt] = (item_id, dept_id, store_id, cat_id, state_id)
#     dico_id[elt] = (cat_id, dept_id, item_id, state_id, store_id)

In [46]:
dico_id['HOBBIES_1_001_CA_1_validation']

('HOBBIES_1_001', 'HOBBIES_1', 'CA_1', 'HOBBIES')

## create dataframe

In [60]:
cols = ['d', 'wm_yr_wk', 
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI']

In [61]:
df = calendar[cols].fillna(-1)

## add prices to dataframe

In [62]:
df.index = df['wm_yr_wk']

dico = {}

for elt in df.columns:
    dico[elt] = df[elt].values

# count = 0
for id1, p in tqdm(prices.groupby('id')):
    p.index = p['wm_yr_wk']
    
    p = p[['sell_price']]
    p.columns = [id1 + '_price']
    
    
    df = df.join(p, how = 'left')
    
    dico[id1 + '_price'] = df[id1 + '_price'].values

    df = df[cols]


In [63]:
a = list(range(50))

In [66]:
len(a[-29:-1])

28

In [ ]:
df = pd.DataFrame(dico)

In [ ]:
df.head()

In [ ]:
save(dico, 'calendar & price')

In [ ]:
dico = load('calendar & price')

In [ ]:
sales.head()

In [ ]:
days = sales.columns[6:].values

In [ ]:
sales.index = sales['id']

In [ ]:
days

In [44]:
sales

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
id,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1_validation,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
HOBBIES_1_002_CA_1_validation,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
HOBBIES_1_003_CA_1_validation,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
HOBBIES_1_004_CA_1_validation,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
HOBBIES_1_005_CA_1_validation,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FOODS_3_823_WI_3_validation,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
FOODS_3_824_WI_3_validation,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
FOODS_3_825_WI_3_validation,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0


In [67]:
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


## add quantity to dataframe

In [ ]:
for elt in tqdm(sales['id'].unique()):
    x = list(sales.loc[elt][days].values)
    
    while len(x) < len(dico['d']):
        x.append(0)
    
    dico[elt + '_quantity'] = x
    

In [ ]:
sales_per_item = sales.groupby(['item_id']).sum()

for elt in tqdm(sales_per_item.index):
    x = list(sales_per_item.loc[elt][days].values)
    
    while len(x) < len(dico['d']):
        x.append(0)
    
    dico[elt + '_quantity'] = x

In [ ]:
sales_per_dept_id = sales.groupby(['dept_id']).sum()

for elt in tqdm(sales_per_dept_id.index):
    x = list(sales_per_dept_id.loc[elt][days].values)
    
    while len(x) < len(dico['d']):
        x.append(0)
    
    dico[elt + '_quantity'] = x

In [ ]:
sales_per_store_id = sales.groupby(['store_id']).sum()

for elt in tqdm(sales_per_store_id.index):
    x = list(sales_per_store_id.loc[elt][days].values)
    
    while len(x) < len(dico['d']):
        x.append(0)
    
    dico[elt + '_quantity'] = x

In [ ]:
sales_per_state_id = sales.groupby(['state_id']).sum()

for elt in tqdm(sales_per_state_id.index):
    x = list(sales_per_state_id.loc[elt][days].values)
    
    while len(x) < len(dico['d']):
        x.append(0)
    
    dico[elt + '_quantity'] = x

In [ ]:
sales_per_cat_id = sales.groupby(['cat_id']).sum() 

for elt in tqdm(sales_per_cat_id.index):
    x = list(sales_per_cat_id.loc[elt][days].values)
    
    while len(x) < len(dico['d']):
        x.append(0)
    
    dico[elt + '_quantity'] = x

In [ ]:
save(dico, 'all_aggregated')

In [ ]:
df = pd.DataFrame(dico)

In [ ]:
del dico

In [ ]:
save(df, 'all_in_one')

## Modeling

In [6]:
df = load('all_in_one')

In [7]:
df.shape

(1969, 64061)

In [8]:
df = df.fillna(-1)

In [9]:
df.head()

,d,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,FOODS_1_001_CA_1_validation_price,...,TX_3_quantity,WI_1_quantity,WI_2_quantity,WI_3_quantity,CA_quantity,TX_quantity,WI_quantity,FOODS_quantity,HOBBIES_quantity,HOUSEHOLD_quantity
0,d_1,11101,-1,-1,-1,-1,0,0,0,2.0,...,3030,2704,2256,4038,14195,9438,8998,23178,3764,5689
1,d_2,11101,-1,-1,-1,-1,0,0,0,2.0,...,3006,2194,1922,4198,13805,9630,8314,22758,3357,5634
2,d_3,11101,-1,-1,-1,-1,0,0,0,2.0,...,2225,1562,2018,3317,10108,6778,6897,17174,2682,3927
3,d_4,11101,-1,-1,-1,-1,1,1,0,2.0,...,2169,1251,2522,3211,11047,7381,6984,18878,2669,3865
4,d_5,11101,-1,-1,-1,-1,1,0,1,2.0,...,1726,2,1175,2132,9925,5912,3309,14603,1814,2729


In [10]:
def categorise(x):
    a = x.unique()
    
    dic = {}
    
    for i, elt in enumerate(a):
        dic[elt] = i
    
    x1 = []
    for elt in x:
        x1.append(dic[elt])
    
    return x1, dic
    

In [11]:
df['event_name_1'], dic_event_name_1 = categorise(df['event_name_1'])
df['event_name_2'], dic_event_name_2 = categorise(df['event_name_2'])
df['event_type_1'], dic_event_type_1 = categorise(df['event_type_1'])
df['event_type_2'], dic_event_type_2 = categorise(df['event_type_2'])

In [12]:
df.head()

,d,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,FOODS_1_001_CA_1_validation_price,...,TX_3_quantity,WI_1_quantity,WI_2_quantity,WI_3_quantity,CA_quantity,TX_quantity,WI_quantity,FOODS_quantity,HOBBIES_quantity,HOUSEHOLD_quantity
0,d_1,11101,0,0,0,0,0,0,0,2.0,...,3030,2704,2256,4038,14195,9438,8998,23178,3764,5689
1,d_2,11101,0,0,0,0,0,0,0,2.0,...,3006,2194,1922,4198,13805,9630,8314,22758,3357,5634
2,d_3,11101,0,0,0,0,0,0,0,2.0,...,2225,1562,2018,3317,10108,6778,6897,17174,2682,3927
3,d_4,11101,0,0,0,0,1,1,0,2.0,...,2169,1251,2522,3211,11047,7381,6984,18878,2669,3865
4,d_5,11101,0,0,0,0,1,0,1,2.0,...,1726,2,1175,2132,9925,5912,3309,14603,1814,2729


In [13]:
'id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1'

('id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1')

In [14]:
df['event_name_1'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
      dtype=int64)

In [16]:
df.head()

,d,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,FOODS_1_001_CA_1_validation_price,...,TX_3_quantity,WI_1_quantity,WI_2_quantity,WI_3_quantity,CA_quantity,TX_quantity,WI_quantity,FOODS_quantity,HOBBIES_quantity,HOUSEHOLD_quantity
0,d_1,11101,0,0,0,0,0,0,0,2.0,...,3030,2704,2256,4038,14195,9438,8998,23178,3764,5689
1,d_2,11101,0,0,0,0,0,0,0,2.0,...,3006,2194,1922,4198,13805,9630,8314,22758,3357,5634
2,d_3,11101,0,0,0,0,0,0,0,2.0,...,2225,1562,2018,3317,10108,6778,6897,17174,2682,3927
3,d_4,11101,0,0,0,0,1,1,0,2.0,...,2169,1251,2522,3211,11047,7381,6984,18878,2669,3865
4,d_5,11101,0,0,0,0,1,0,1,2.0,...,1726,2,1175,2132,9925,5912,3309,14603,1814,2729


In [28]:
list(df.columns)[10000:20000]

['FOODS_3_387_CA_2_validation_price',
 'FOODS_3_387_CA_3_validation_price',
 'FOODS_3_387_CA_4_validation_price',
 'FOODS_3_387_TX_1_validation_price',
 'FOODS_3_387_TX_2_validation_price',
 'FOODS_3_387_TX_3_validation_price',
 'FOODS_3_387_WI_1_validation_price',
 'FOODS_3_387_WI_2_validation_price',
 'FOODS_3_387_WI_3_validation_price',
 'FOODS_3_388_CA_1_validation_price',
 'FOODS_3_388_CA_2_validation_price',
 'FOODS_3_388_CA_3_validation_price',
 'FOODS_3_388_CA_4_validation_price',
 'FOODS_3_388_TX_1_validation_price',
 'FOODS_3_388_TX_2_validation_price',
 'FOODS_3_388_TX_3_validation_price',
 'FOODS_3_388_WI_1_validation_price',
 'FOODS_3_388_WI_2_validation_price',
 'FOODS_3_388_WI_3_validation_price',
 'FOODS_3_389_CA_1_validation_price',
 'FOODS_3_389_CA_2_validation_price',
 'FOODS_3_389_CA_3_validation_price',
 'FOODS_3_389_CA_4_validation_price',
 'FOODS_3_389_TX_1_validation_price',
 'FOODS_3_389_TX_2_validation_price',
 'FOODS_3_389_TX_3_validation_price',
 'FOODS_3_38

In [36]:
len(dico_id.keys())

30490

In [19]:
df.tail()

,d,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,FOODS_1_001_CA_1_validation_price,...,TX_3_quantity,WI_1_quantity,WI_2_quantity,WI_3_quantity,CA_quantity,TX_quantity,WI_quantity,FOODS_quantity,HOBBIES_quantity,HOUSEHOLD_quantity
1964,d_1965,11620,0,0,0,0,0,1,1,2.24,...,0,0,0,0,0,0,0,0,0,0
1965,d_1966,11620,0,0,0,0,0,0,0,2.24,...,0,0,0,0,0,0,0,0,0,0
1966,d_1967,11620,0,0,0,0,0,0,0,2.24,...,0,0,0,0,0,0,0,0,0,0
1967,d_1968,11621,0,0,0,0,0,0,0,2.24,...,0,0,0,0,0,0,0,0,0,0
1968,d_1969,11621,14,1,4,1,0,0,0,2.24,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df['d']

0          d_1
1          d_2
2          d_3
3          d_4
4          d_5
         ...  
1964    d_1965
1965    d_1966
1966    d_1967
1967    d_1968
1968    d_1969
Name: d, Length: 1969, dtype: object

In [ ]:
train_len = 150
def build_example(df, id1, row , is_train = True):
    item_id, dept_id, store_id, cat_id = dico_id[id1]
    
    before = df.iloc[row - train_len:row]
    after = df.iloc[row : row + 28]
    
    ## Cat data
    event_name_1 = before['event_name_1'].values
    event_name_2 = before['event_name_2'].values
    event_type_1 = before['event_type_1'].values
    event_type_2 = before['event_type_2'].values
    
    cat = np.array([event_name_1, event_name_2, event_type_1, event_type_2])
    
    event_name_1_post = after['event_name_1'].values
    event_name_2_post = after['event_name_2'].values
    event_type_1_post = after['event_type_1'].values
    event_type_2_post = after['event_type_2'].values
    
    cat_post = np.array([event_name_1_post, event_name_2_post, event_type_1_post, event_type_2_post])
    
    ## Numerical_data
    
    price = before[id1+'_price'].values
    price_post = after[id1+'_price'].values
    
    q = before[id1 + '_quantity']
    q_item = before[item_id +  '_quantity']
    q_dept = before[dept_id +  '_quantity']
    q_store = before[store_id +  '_quantity']
    q_cat = before[cat_id + '_quantity']
    
    
    snap_CA = before['snap_CA'].values
    snap_TX = before['snap_TX'].values
    snap_WI = before['snap_WI'].values
    
    snap_CA_post = after['snap_CA'].values
    snap_TX_post = after['snap_TX'].values
    snap_WI_post = after['snap_WI'].values
    
    
    pre = np.array([event_name_1, event_name_2, event_type_1, event_type_2, price, q, q_item, q_dept, q_store, q_cat, snap_CA, snap_TX, snap_WI])
    
    post = np.array([event_name_1_post, event_name_2_post, event_type_1_post, event_type_2_post, price_post,snap_CA_post,snap_TX_post,snap_WI_post])
        
    if is_train:
        target = after[id1 + '_quantity'].values
    else:
        target = None
        
    del before
    del after
    
    return pre, post, target

In [34]:
sales

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
id,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1_validation,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
HOBBIES_1_002_CA_1_validation,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
HOBBIES_1_003_CA_1_validation,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
HOBBIES_1_004_CA_1_validation,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
HOBBIES_1_005_CA_1_validation,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FOODS_3_823_WI_3_validation,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
FOODS_3_824_WI_3_validation,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
FOODS_3_825_WI_3_validation,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0


In [ ]:
for i in range(100):
    n_samples = 1000
    X_train_pre = list(np.zeros(n_samples))
    X_train_post = list(np.zeros(n_samples))
    y_train = list(np.zeros(n_samples))

    ids = sales['id'].unique()
    r = list(range(train_len+5, 1884))

    for elt in tqdm(range(n_samples)):
        id1 = random.choice(ids)
        row = random.choice(r)

        pre, post,  target = build_example(df, id1, row)
        gc.collect()
        X_train_pre[elt] = pre.T
        X_train_post[elt] = post.T
        y_train[elt] = target

    X_train_pre = np.array(X_train_pre)
    X_train_post = np.array(X_train_post)
    y_train = np.array(y_train)
    gc.collect()
    save((X_train_pre, X_train_post, y_train), 'batch_'+str(i+13), 'batch')

In [ ]:
save((X_train_pre, X_train_post, y_train), 'batch_3', 'batch')

In [ ]:
(X_train_pre, X_train_post, y_train) = load('batch_1', 'batch')

In [ ]:
n_samples = 5000

X_test_pre = list(np.zeros(n_samples))
X_test_post = list(np.zeros(n_samples))
y_test = list(np.zeros(n_samples))

ids = sales['id'].unique()
r = list(range(train_len+5, 1884))

for elt in tqdm(range(n_samples)):
    id1 = random.choice(ids)
    row = random.choice(r)
    
    pre, post,  target = build_example(df, id1, row)
    
    X_test_pre[elt] = pre.T
    X_test_post[elt] = post.T
    y_test[elt] = target

X_test_pre = np.array(X_test_pre)
X_test_post = np.array(X_test_post)
y_test = np.array(y_test)


In [ ]:
save((X_train_pre, X_train_post, y_train), 'batch_1_test', 'batch')

In [3]:
X_train_pre, X_train_post, y_train = load('batch_1', 'batch')

In [ ]:
X_train_pre = list(np.zeros(80))
X_train_post = list(np.zeros(80))
y_train = list(np.zeros(80))

for i in range(1,81):
    x, x1, y = load('batch_'+str(i), 'batch')
    
    X_train_pre[i-1] = x
    X_train_post[i-1] = x1
    y_train[i-1] = y
    
X_train_pre = np.concatenate(X_train_pre, axis = 0)
X_train_post = np.concatenate(X_train_post, axis = 0)
y_train = np.concatenate(y_train, axis = 0)

In [ ]:
X_test_pre = list(np.zeros(11))
X_test_post = list(np.zeros(11))
y_test = list(np.zeros(11))

for i in range(80,91):
    x, x1, y = load('batch_'+str(i), 'batch')
    X_test_pre[i-80] = x
    X_test_post[i-80] = x1
    y_test[i-80] = y
    
X_test_pre = np.concatenate(X_test_pre, axis = 0)
X_test_post = np.concatenate(X_test_post, axis = 0)
y_test = np.concatenate(y_test, axis = 0)

In [ ]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dropout, Bidirectional,RepeatVector, Dense, Flatten, Embedding, TimeDistributed, Concatenate, LSTM, BatchNormalization, Lambda, Reshape
from tensorflow.keras.regularizers import l2


# num = Input(shape = (train_len, 9))
# event_name_1 = Input(shape = (train_len, 1))
# event_name_2 = Input(shape = (train_len, 1))
# event_type_1 = Input(shape = (train_len, 1))
# event_type_2 = Input(shape = (train_len, 1))
# pre = [event_name_1, event_name_2, event_type_1, event_type_2, num]

# num_post = Input(shape = (28, 4))
# event_name_1_post = Input(shape = (28, 1))
# event_name_2_post = Input(shape = (28, 1))
# event_type_1_post = Input(shape = (28, 1))
# event_type_2_post = Input(shape = (28, 1))
# post = [event_name_1_post, event_name_2_post, event_type_1_post, event_type_2_post, num_post]

pre = Input(shape = (train_len, 13))
post = Input(shape = (28, 8))


num = pre[:,:,4:]
event_name_1 = pre[:,:,0]
event_name_2 = pre[:,:,1]
event_type_1 = pre[:,:,2]
event_type_2 = pre[:,:,3]
# pre = [event_name_1, event_name_2, event_type_1, event_type_2, num]

num_post = post[:,:,4:]
event_name_1_post = post[:,:,0]
event_name_2_post = post[:,:,1]
event_type_1_post = post[:,:,2]
event_type_2_post = post[:,:,3]


inputs = [pre, post]

## Numerical Layers
num_emb = TimeDistributed(Dense(16))(num)
num_emb_post = TimeDistributed(Dense(16))(num_post)

## Embeddings Categorical layers
emb_name_1 = TimeDistributed(Embedding( 32 ,4))
emb_name_2 = TimeDistributed(Embedding( 5 ,2))
emb_type_1 = TimeDistributed(Embedding( 5 ,2))
emb_type_2 = TimeDistributed(Embedding( 3 ,1))

event_name_1_emb = emb_name_1(Reshape((train_len,1))(event_name_1))
event_name_2_emb = emb_name_2(Reshape((train_len,1))(event_name_2))
event_type_1_emb = emb_type_1(Reshape((train_len,1))(event_type_1))
event_type_2_emb = emb_type_2(Reshape((train_len,1))(event_type_2))

event_name_1_emb_post = emb_name_1(Reshape((28,1))(event_name_1_post))
event_name_2_emb_post = emb_name_2(Reshape((28,1))(event_name_2_post))
event_type_1_emb_post = emb_type_1(Reshape((28,1))(event_type_1_post))
event_type_2_emb_post = emb_type_2(Reshape((28,1))(event_type_2_post))

event_name_1_emb = Reshape((train_len,4))(event_name_1_emb)
event_name_2_emb = Reshape((train_len,2))(event_name_2_emb)
event_type_1_emb = Reshape((train_len,2))(event_type_1_emb)
event_type_2_emb = Reshape((train_len,1))(event_type_2_emb)

event_name_1_emb_post = Reshape((28,4))(event_name_1_emb_post)
event_name_2_emb_post = Reshape((28,2))(event_name_2_emb_post)
event_type_1_emb_post = Reshape((28,2))(event_type_1_emb_post)
event_type_2_emb_post = Reshape((28,1))(event_type_2_emb_post)

embedded_pre = [event_name_1_emb, event_name_2_emb, event_type_1_emb, event_type_2_emb]
embedded_post = [event_name_1_emb_post, event_name_2_emb_post, event_type_1_emb_post, event_type_2_emb_post]

merged = Concatenate()(embedded_pre + [num_emb])
merged_post = Concatenate()(embedded_post + [num_emb_post])

merged = BatchNormalization()(merged)

merged = TimeDistributed(Dense(64, activation = 'relu'))(merged)
merged = TimeDistributed(Dense(64, activation = 'relu'))(merged)
merged = TimeDistributed(Dense(64, activation = 'relu'))(merged)
merged = TimeDistributed(Dense(64, activation = 'relu'))(merged)

# lstm_out = Bidirectional(LSTM(64, activation = 'tanh', return_sequences = True))(merged)
# lstm_out = Bidirectional(LSTM(64, activation = 'tanh', return_sequences = True))(lstm_out)
lstm_out = Bidirectional(LSTM(64, activation = 'tanh', return_sequences = True))(merged)
lstm_out = LSTM(64, activation = 'tanh', return_sequences = False)(lstm_out)

lstm_out = RepeatVector(28)(lstm_out)

out = Concatenate()([merged_post, lstm_out])

out = LSTM(64, activation = 'tanh', return_sequences = True)(out)
out = LSTM(64, activation = 'tanh', return_sequences = True)(out)
out = LSTM(64, activation = 'tanh', return_sequences = True)(out)
out = LSTM(64, activation = 'tanh', return_sequences = True)(out)


out = TimeDistributed(Dense(1, activation = 'linear'))(out)

model = Model(inputs , out)

In [ ]:
model.summary()

In [ ]:
X_train_pre.shape

In [ ]:
X_test_post.shape

In [ ]:
from tensorflow.keras.optimizers import SGD

optimizer = SGD(0.001)
model.compile(loss='mse',
              optimizer=optimizer)

batch_size = 16
epochs = 50

history = model.fit([X_train_pre, X_train_post], y_train, batch_size = batch_size, epochs = epochs, validation_data=([X_test_pre, X_test_post], y_test))

In [ ]:
a = model.predict([X_test_pre, X_test_post])

In [ ]:
a = a.reshape(a.shape[0], a.shape[1])

In [ ]:
a

In [ ]:
y_test